# Advent of Code 2021 - Day 25

In [141]:
sealed interface Tile {
    val character: String
    object Empty : Tile { override val character = "." }
    sealed interface Cucumber : Tile {
        object East : Cucumber { override val character = ">" }
        object South : Cucumber { override val character = "v" }
    }
}

fun Char.toTile(): Tile = when(this) {
    '>' -> Tile.Cucumber.East
    'v' -> Tile.Cucumber.South
    else -> Tile.Empty
}

typealias Ocean = List<List<Tile>>
typealias MutableOcean = MutableList<MutableList<Tile>>
fun Ocean.toMutableOcean(): MutableOcean = map(List<Tile>::toMutableList).toMutableList()

data class Index(val row: Int, val col: Int)

In [142]:
import java.io.File

val input: Ocean = File("Day25.input.txt")
    .bufferedReader()
    .lineSequence()
    .map { row -> row.map { it.toTile() } }
    .toList()

## Part 1



In [143]:
operator fun Ocean.get(index: Index): Tile = this[index.row][index.col]
operator fun MutableOcean.set(index: Index, tile: Tile) { this[index.row][index.col] = tile }

fun Index.right(ocean: Ocean): Index = if (col + 1 < ocean[row].size) Index(row, col + 1) else Index(row, 0)
fun Index.down(ocean: Ocean): Index = if (row + 1 < ocean.size) Index(row + 1, col) else Index(0, col)

fun Index.move(ocean: Ocean): Index? = when (ocean[row][col]) {
    Tile.Empty -> null
    Tile.Cucumber.East -> right(ocean).let { if (ocean[it] == Tile.Empty) it else null }
    Tile.Cucumber.South -> down(ocean).let { if (ocean[it] == Tile.Empty) it else null }
}

data class Move(val current: Index, val next: Index)
fun Ocean.moves(tile: Tile): List<Move> = foldIndexed(mutableListOf()) { r, outer, row ->
    row.foldIndexed(outer) { c, inner, it ->
        val current = Index(r, c)
        if (it == tile) current.move(this)?.let { inner.add(Move(current, it)) }
        inner
    }
}

fun Ocean.move(tile: Tile): Ocean = moves(tile).fold(toMutableOcean()) { acc, (current, next) ->
    acc.apply {
        set(current, Tile.Empty)
        set(next, tile)
    }
}

fun Ocean.moveWhile(condition: (Ocean) -> Boolean, step: Int = 0): Pair<Ocean, Int> =
    if (!condition(this)) this to step
    else move(Tile.Cucumber.East).move(Tile.Cucumber.South).moveWhile(condition, step + 1)

input.moveWhile({ (it.moves(Tile.Cucumber.East) + it.moves(Tile.Cucumber.South)).isNotEmpty() }).second + 1

482

### Notes



## Part 2



In [144]:
// Part 2

### Notes

